In [1]:
import pandas as pd
import time
from collections import Counter

from scipy import sparse
import lightgbm as lgb

from sklearn.preprocessing import OneHotEncoder,LabelEncoder

from sklearn.model_selection import train_test_split 
import gc

In [31]:
order_train = pd.read_csv('./data/train/ord_train.csv')
order_chaifen = pd.read_csv('./data/train/ord_chaifen.csv')
order_bkroomstatus = pd.read_csv('./data/train/ord_bkroomstatus.csv')
order_zqroomstatus = pd.read_csv('./data/train/ord_zqroomstatus.csv')


hotelinfo = pd.read_csv('./data/train/hotelinfo.csv')
mroominfp = pd.read_csv('./data/train/mroominfo.csv')
mhotelinfo = pd.read_csv('./data/train/mhotelinfo.csv')

testa = pd.read_csv('./data/test/ord_testA.csv', encoding='gb2312')
testb = pd.read_csv('./data/test/ord_testB.csv', encoding='gb2312')

In [32]:
test = pd.concat([testa, testb])

test['noroom'] = -1
data = pd.concat([order_train, test])

In [33]:
len(data[data.noroom == 1])/len(data)

0.05085908902917736

In [34]:
data.head()

,arrival,city,commission,confirmdate,countryid,etd,hotel,hotelbelongto,hotelstar,isebookinghtl,...,ordadvanceday,orderdate,orderid,orderstatus,ordroomnum,price,room,supplierchannel,supplierid,zone
0,2017-06-23,1777,57.60000,NaN,111,2017-06-25,6459206,SHT,5,T,...,40,2017-05-15 10:03:34,3678052984,C,1.0,717.6,62786796,合作,1897.0,3302.0
1,2017-05-05,622,7.98600,2017-05-02 21:25:45,4,2017-05-06,4515821,SHT,4,T,...,3,2017-05-02 21:25:37,3623712471,S,1.0,99.0,51183791,合作,2322.0,NaN
2,2017-07-01,366,89.71200,2017-06-20 15:03:53,30,2017-07-04,3403530,SHT,5,F,...,12,2017-06-20 15:03:39,3935430746,S,1.0,1118.1,18594989,合作,3686.0,NaN
3,2017-08-22,274,22.45260,2017-07-19 23:45:14,42,2017-08-24,4984070,HTL,5,T,...,34,2017-07-19 23:45:12,4095204178,S,1.0,250.5,72423477,直签,2299.0,705.0
4,2017-05-27,723,17.44395,NaN,108,2017-05-28,3796626,SHT,5,T,...,1,2017-05-26 16:21:36,3753007353,C,1.0,192.3,55656893,合作,2391.0,340.0


In [35]:
import gc
gc.collect()

422

In [36]:
# 
def cal_time_stamp(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d'))
    #day = time.localtime(day)
    return day

data['time_arrival'] = data.arrival.apply(cal_time_stamp)

In [37]:
def cal_month(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d'))
    day = time.localtime(day)
    return day.tm_mon

data['month_arrival'] = data.arrival.apply(cal_month)

In [38]:
def cal_week(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d'))
    day = time.localtime(day)
    return day.tm_wday

data['week_arrival'] = data.arrival.apply(cal_week)

In [39]:
data.head()

,arrival,city,commission,confirmdate,countryid,etd,hotel,hotelbelongto,hotelstar,isebookinghtl,...,orderstatus,ordroomnum,price,room,supplierchannel,supplierid,zone,time_arrival,month_arrival,week_arrival
0,2017-06-23,1777,57.60000,NaN,111,2017-06-25,6459206,SHT,5,T,...,C,1.0,717.6,62786796,合作,1897.0,3302.0,1.498147e+09,6,4
1,2017-05-05,622,7.98600,2017-05-02 21:25:45,4,2017-05-06,4515821,SHT,4,T,...,S,1.0,99.0,51183791,合作,2322.0,NaN,1.493914e+09,5,4
2,2017-07-01,366,89.71200,2017-06-20 15:03:53,30,2017-07-04,3403530,SHT,5,F,...,S,1.0,1118.1,18594989,合作,3686.0,NaN,1.498838e+09,7,5
3,2017-08-22,274,22.45260,2017-07-19 23:45:14,42,2017-08-24,4984070,HTL,5,T,...,S,1.0,250.5,72423477,直签,2299.0,705.0,1.503331e+09,8,1
4,2017-05-27,723,17.44395,NaN,108,2017-05-28,3796626,SHT,5,T,...,C,1.0,192.3,55656893,合作,2391.0,340.0,1.495814e+09,5,5


In [40]:
# hour
def cal_orderdate_hour(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d %H:%M'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d %H:%M:%S'))
    day = time.localtime(day)
    return day.tm_hour

data['orderdate_hour'] = data.orderdate.apply(cal_orderdate_hour)

In [41]:
data = data.fillna(-1)

In [42]:
data.columns

Index(['arrival', 'city', 'commission', 'confirmdate', 'countryid', 'etd',
       'hotel', 'hotelbelongto', 'hotelstar', 'isebookinghtl', 'isholdroom',
       'isvendor', 'masterbasicroomid', 'masterhotelid', 'noroom',
       'ordadvanceday', 'orderdate', 'orderid', 'orderstatus', 'ordroomnum',
       'price', 'room', 'supplierchannel', 'supplierid', 'zone',
       'time_arrival', 'month_arrival', 'week_arrival', 'orderdate_hour'],
      dtype='object')

In [43]:
gc.collect()

19

In [11]:
data[data.noroom == -1]['room']

4023086    71340604
4023087    70578904
4023088    91096440
4023089    20487676
4023090    88163053
4023091    57665150
4023092     5255520
4023093    80986427
4023094    53374494
4023095    30032626
4023096     6647651
4023097     4386059
4023098    56364833
4023099    56860151
4023100    70832568
4023101    78128668
4023102    73593346
4023103    65978769
4023104    85292361
4023105    64080594
4023106    20864490
4023107    63383128
4023108    70529309
4023109     8384766
4023110     8570317
4023111    70400651
4023112    12686091
4023113    71804191
4023114    78937585
4023115    88144160
             ...   
4045127    50625874
4045128    70418058
4045129     9951927
4045130    78633071
4045131    27439415
4045132    83036943
4045133    91306693
4045134    41232906
4045135    83286531
4045136    70606225
4045137    15516585
4045138     7498468
4045139    72704714
4045140    63784888
4045141    48114834
4045142    85684232
4045143    31042086
4045144    19969205
4045145    41879961


In [3]:
data = pd.read_csv('./data1.csv')

/home/xiaolian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train = data[data.noroom != -1]

In [4]:
data['eta_arrival'] = pd.to_datetime(data.etd) - pd.to_datetime(data.arrival)

In [5]:
data.eta_arrival = data.eta_arrival.astype(int) / (3600*24*1000000000)

In [6]:
data.eta_arrival = data.eta_arrival.astype(int)

In [7]:
group = train[['ordadvanceday', 'noroom']].groupby('ordadvanceday')['noroom'].mean()
# group = dict(group)

In [8]:
data['cvr_ordadvanceday'] = data.ordadvanceday.apply(lambda x :group[x])

In [9]:
group = train[['week_arrival', 'noroom']].groupby('week_arrival')['noroom'].mean()

In [10]:
group

week_arrival
0    0.044202
1    0.047093
2    0.050121
3    0.051923
4    0.055159
5    0.061394
6    0.046100
Name: noroom, dtype: float64

In [11]:
data['cvr_week_arrival'] = data.week_arrival.apply(lambda x :group[x])

In [12]:
del train
gc.collect()

27

In [13]:
# arrival day
def cal_arrival_day(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d'))
    day = time.localtime(day)
    return day.tm_mday

data['arrival_day'] = data.arrival.apply(cal_arrival_day)

In [14]:
# orderdate day
def cal_orderdate_day(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d %H:%M'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d %H:%M:%S'))
    day = time.localtime(day)
    return day.tm_mday

data['orderdate_day'] = data.orderdate.apply(cal_orderdate_day)

In [15]:
# orderdate mouth
def cal_orderdate_mouth(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d %H:%M'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d %H:%M:%S'))
    day = time.localtime(day)
    return day.tm_mon

data['orderdate_mouth'] = data.orderdate.apply(cal_orderdate_mouth)

In [16]:
# orderdate week
def cal_orderdate_week(day):
    #day = day.split(' ')[0]
    #print(day)
    try:
        day = time.mktime(time.strptime(day, '%Y/%m/%d %H:%M'))
    except:
        day = time.mktime(time.strptime(day, '%Y-%m-%d %H:%M:%S'))
    day = time.localtime(day)
    return day.tm_wday

data['orderdate_week'] = data.orderdate.apply(cal_orderdate_week)

In [17]:
data.columns

Index(['arrival', 'city', 'commission', 'confirmdate', 'countryid', 'etd',
       'hotel', 'hotelbelongto', 'hotelstar', 'isebookinghtl', 'isholdroom',
       'isvendor', 'masterbasicroomid', 'masterhotelid', 'noroom',
       'ordadvanceday', 'orderdate', 'orderid', 'orderstatus', 'ordroomnum',
       'price', 'room', 'supplierchannel', 'supplierid', 'zone',
       'time_arrival', 'month_arrival', 'week_arrival', 'orderdate_hour',
       'city_count', 'countryid_count', 'hotel_count', 'zone_count',
       'room_count', 'isholdroom_count', 'ordadvanceday_count',
       'arrival_count', 'etd_count', 'masterbasicroomid_count',
       'masterhotelid_count', 'supplierid_count', 'isvendor_count',
       'hotelbelongto_count', 'isebookinghtl_count', 'hotelstar_count',
       'supplierchannel_count', 'time_arrival_count', 'month_arrival_count',
       'week_arrival_count', 'orderdate_hour_count', 'eta_arrival',
       'cvr_ordadvanceday', 'cvr_week_arrival', 'arrival_day', 'orderdate_day',
   

In [21]:
# custom
# 11
customs = [
           # row
          'arrival',
          'etd',
          'ordadvanceday',
           # arrival mouth week day
          'month_arrival', 
          'week_arrival', 
          'arrival_day',
           # orderdate mouth week day hour
            'orderdate_hour',
            'orderdate_day',
            'orderdate_mouth',
            'orderdate_week',
          # start -end
          'eta_arrival',
           
         ]
# 14
hotels = ['hotelstar',
          'city',
          'countryid',
          'hotel',
          'zone',
          'room',
          'isholdroom',
           'masterbasicroomid',
          'masterhotelid',
          'supplierid',
          'isvendor',
           'hotelbelongto',
          'isebookinghtl',
          'supplierchannel']

for custom in customs:
    for hotel in hotels:
        start = time.time()
        new_feature = custom + '_' + hotel
        data[new_feature] = data[custom].astype(str) + '_' + data[hotel].astype(str)
        data[new_feature] = LabelEncoder().fit_transform(data[new_feature])
        print(custom, ' ', hotel, ' finish!', time.time() - start)

arrival   hotelstar  finish! 7.800585031509399
arrival   city  finish! 8.287858009338379
arrival   countryid  finish! 8.23425030708313
arrival   hotel  finish! 8.721322298049927
arrival   zone  finish! 7.63059139251709
arrival   room  finish! 8.68870735168457
arrival   isholdroom  finish! 6.28204607963562
arrival   masterbasicroomid  finish! 8.23661208152771
arrival   masterhotelid  finish! 8.531030654907227
arrival   supplierid  finish! 7.6264190673828125
arrival   isvendor  finish! 7.62387752532959
arrival   hotelbelongto  finish! 6.4503889083862305
arrival   isebookinghtl  finish! 6.487075328826904
arrival   supplierchannel  finish! 8.573465347290039
etd   hotelstar  finish! 7.77666711807251
etd   city  finish! 8.243236064910889
etd   countryid  finish! 8.289733648300171
etd   hotel  finish! 8.813609838485718
etd   zone  finish! 7.6060755252838135
etd   room  finish! 8.648338079452515
etd   isholdroom  finish! 6.469723463058472
etd   masterbasicroomid  finish! 8.394716501235962
etd 

In [64]:
hotelinfo = hotelinfo[['hotel', 'totalrooms']]

In [65]:
order_roomstatus.head()

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276
2,8573004,2017-08-10,N,2017-08-07 18:25:40,7769214,538682,688203
3,13619122,2017-08-22,N,2017-08-05 09:44:34,5787108,785102,1074098
4,39616190,2017-08-13,N,2017-08-05 19:15:17,18215309,5114310,5086203


In [66]:
order_roomstatus = pd.merge(order_roomstatus, hotelinfo, on = 'hotel', how = 'left')

In [67]:
order_roomstatus.head()

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid,totalrooms
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089,24.0
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276,34.0
2,8573004,2017-08-10,N,2017-08-07 18:25:40,7769214,538682,688203,195.0
3,13619122,2017-08-22,N,2017-08-05 09:44:34,5787108,785102,1074098,26.0
4,39616190,2017-08-13,N,2017-08-05 19:15:17,18215309,5114310,5086203,25.0


In [68]:
order_roomstatus.rename(columns = {'totalrooms':'hotel_totalrooms'}, inplace = True)

In [69]:
order_roomstatus.head()

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid,hotel_totalrooms
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089,24.0
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276,34.0
2,8573004,2017-08-10,N,2017-08-07 18:25:40,7769214,538682,688203,195.0
3,13619122,2017-08-22,N,2017-08-05 09:44:34,5787108,785102,1074098,26.0
4,39616190,2017-08-13,N,2017-08-05 19:15:17,18215309,5114310,5086203,25.0


In [70]:
mroominfp.rename(columns = {'totalrooms':'mroom_totalrooms'}, inplace=True)

/home/xiaolian/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [71]:
mroominfp = mroominfp[['masterbasicroomid', 'mroom_totalrooms']]

In [72]:
mroominfp.head()

,masterbasicroomid,mroom_totalrooms
0,10023629,1
1,10030510,3
2,10031578,2
3,10032516,2
4,10034229,588


In [73]:
order_roomstatus = pd.merge(order_roomstatus, mroominfp, on = 'masterbasicroomid', how = 'left')

In [74]:
order_roomstatus

,room,arrival,roomstatus,createtime,masterbasicroomid,hotel,masterhotelid,hotel_totalrooms,mroom_totalrooms
0,49359351,2017-10-01,N,2017-08-24 03:54:17,44039493,4076704,1857089,24.0,49.0
1,13037426,2017-08-25,N,2017-08-05 14:34:38,7924733,229256,699276,34.0,155.0
2,8573004,2017-08-10,N,2017-08-07 18:25:40,7769214,538682,688203,195.0,1394.0
3,13619122,2017-08-22,N,2017-08-05 09:44:34,5787108,785102,1074098,26.0,242.0
4,39616190,2017-08-13,N,2017-08-05 19:15:17,18215309,5114310,5086203,25.0,312.0
5,2253912,2017-08-08,N,2017-08-06 05:08:34,5786834,178623,712938,42.0,210.0
6,12494010,2017-09-07,N,2017-09-02 02:33:47,3692832,228490,994106,202.0,797.0
7,13402474,2017-08-09,N,2017-08-07 08:07:14,64670476,693978,3692906,50.0,91.0
8,15922097,2017-09-16,N,2017-09-14 17:43:18,66342571,229190,698813,7.0,49.0
9,63495756,2017-09-02,N,2017-08-06 09:54:27,35202195,7312322,6123458,21.0,1.0


In [75]:
mhotelinfo.head()

,masterhotelid,glon,glat,star,zone,city,country
0,8657540,151.346340,-33.516495,0,NaN,4129,15
1,10095686,92.526480,58.190506,0,NaN,71743,30
2,15935739,34.558693,49.588090,0,NaN,5615,119
3,15156172,110.363140,-7.771237,2,0.0,741,108
4,10950378,20.406840,44.806694,0,2298.0,10257,183


In [76]:
mhotelinfo = mhotelinfo[['masterhotelid', 'glon', 'glat']]

In [77]:
order_roomstatus = pd.merge(order_roomstatus, mhotelinfo, on = 'masterhotelid', how = 'left')

In [78]:
order_roomstatus = order_roomstatus.drop_duplicates(['room', 'arrival'])


In [88]:
order_roomstatus.columns

Index(['room', 'arrival', 'roomstatus', 'createtime', 'masterbasicroomid',
       'hotel', 'masterhotelid', 'hotel_totalrooms', 'mroom_totalrooms',
       'glon', 'glat'],
      dtype='object')

In [89]:
order_roomstatus = order_roomstatus[['room', 'arrival','roomstatus',
        'hotel_totalrooms', 'mroom_totalrooms',
       'glon', 'glat' ]]

In [91]:
data = pd.merge(data, order_roomstatus, on = ['room', 'arrival'], how = 'left')

In [116]:
data.head()

,arrival,city,commission,confirmdate,countryid,etd,hotel,hotelbelongto,hotelstar,isebookinghtl,...,eta_arrival_supplierid,eta_arrival_isvendor,eta_arrival_hotelbelongto,eta_arrival_isebookinghtl,eta_arrival_supplierchannel,roomstatus,hotel_totalrooms,mroom_totalrooms,glon,glat
0,142,481,57.60000,-1,99,143,223100,3,5,1,...,2165,43,63,44,111,0,-1.0,-1.0,-1.00000,-1.00000
1,93,76,7.98600,2017-05-02 21:25:45,3,93,179653,3,4,1,...,1163,23,34,23,60,0,-1.0,-1.0,-1.00000,-1.00000
2,150,54,89.71200,2017-06-20 15:03:53,25,152,155301,3,5,0,...,3088,50,71,49,121,0,-1.0,-1.0,-1.00000,-1.00000
3,202,31,22.45260,2017-07-19 23:45:14,37,203,185949,1,5,1,...,2195,43,61,44,112,3,166.0,317.0,126.97841,37.55841
4,115,132,17.44395,-1,96,115,169278,3,5,1,...,1204,22,34,23,60,0,-1.0,-1.0,-1.00000,-1.00000


In [2]:
data = pd.read_csv('./data1.csv')

/home/xiaolian/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# custom
# 11
customs = [
    # row
    'arrival',
          'etd',
          'ordadvanceday',
           # arrival mouth week day
          'month_arrival', 
        'week_arrival', 
           'arrival_day',
           # orderdate mouth week day hour
        'orderdate_hour',
        'orderdate_day',
       'orderdate_mouth',
        'orderdate_week',
           # start -end
         'eta_arrival',
           
         ]
# 14
hotels = ['hotelstar',
          'city',
          'countryid',
          'hotel',
          'zone',
          'room',
          'isholdroom',
           'masterbasicroomid',
          'masterhotelid',
          'supplierid',
          'isvendor',
           'hotelbelongto',
          'isebookinghtl',
          'supplierchannel']
#统计次数
for custom in customs:
    for hotel in hotels:
        start = time.time()
        new_feature = custom + '_' + hotel
        new_feature_count = new_feature + '_count'
        count_train = Counter(data[new_feature])
        #count_test = Counter(data[data.noroom == -1][new_feature])
        data[new_feature_count] = data[new_feature].apply(lambda x: count_train[x])
        # data[new_feature] = LabelEncoder().fit_transform(data[new_feature])
        print(custom, ' ', hotel, ' finish!', time.time() - start)

arrival   hotelstar  finish! 1.3487257957458496
arrival   city  finish! 1.6601567268371582
arrival   countryid  finish! 1.4552216529846191
arrival   hotel  finish! 2.077849864959717
arrival   zone  finish! 1.7298715114593506
arrival   room  finish! 2.01531982421875
arrival   isholdroom  finish! 1.3936381340026855
arrival   masterbasicroomid  finish! 2.173478841781616
arrival   masterhotelid  finish! 2.307926893234253
arrival   supplierid  finish! 1.5802843570709229
arrival   isvendor  finish! 1.3399159908294678
arrival   hotelbelongto  finish! 1.3439102172851562
arrival   isebookinghtl  finish! 1.3406856060028076
arrival   supplierchannel  finish! 1.3686201572418213
etd   hotelstar  finish! 1.345123529434204
etd   city  finish! 1.5941736698150635
etd   countryid  finish! 1.4796767234802246
etd   hotel  finish! 2.1156840324401855
etd   zone  finish! 1.7063651084899902
etd   room  finish! 2.055715799331665
etd   isholdroom  finish! 1.4010586738586426
etd   masterbasicroomid  finish! 2.22

In [4]:
data.head()

,arrival,city,commission,confirmdate,countryid,etd,hotel,hotelbelongto,hotelstar,isebookinghtl,...,eta_arrival_zone_count,eta_arrival_room_count,eta_arrival_isholdroom_count,eta_arrival_masterbasicroomid_count,eta_arrival_masterhotelid_count,eta_arrival_supplierid_count,eta_arrival_isvendor_count,eta_arrival_hotelbelongto_count,eta_arrival_isebookinghtl_count,eta_arrival_supplierchannel_count
0,2017-06-23,1777,57.60000,-1,111,2017-06-25,6459206,SHT,5,T,...,359,10,651836,82,588,26168,584516,527698,829851,577157
1,2017-05-05,622,7.98600,2017-05-02 21:25:45,4,2017-05-06,4515821,SHT,4,T,...,558546,1,1082760,1,22,46917,873747,744915,1205202,770549
2,2017-07-01,366,89.71200,2017-06-20 15:03:53,30,2017-07-04,3403530,SHT,5,F,...,168514,12,383423,20,147,5338,278917,328986,136322,365320
3,2017-08-22,274,22.45260,2017-07-19 23:45:14,42,2017-08-24,4984070,HTL,5,T,...,19483,88,417220,258,702,227340,584516,489356,829851,336875
4,2017-05-27,723,17.44395,-1,108,2017-05-28,3796626,SHT,5,T,...,4249,36,1082760,181,205,591,795545,744915,1205202,770549


In [5]:
for feature in ['month_arrival','eta_arrival']:
    start = time.time()
    new_feature = feature + '_count'
    count_train = Counter(data[feature])
    #count_test = Counter(data[data.noroom == -1][feature])
    data[new_feature] = data[feature].apply(lambda x: count_train[x])
        
    # data[new_feature] = LabelEncoder().fit_transform(data[new_feature])
    #print(custom, ' ', hotel, ' finish!', time.time() - start)

In [6]:
gc.collect()

262

In [7]:
hotelinfo = pd.read_csv('./data/train/hotelinfo.csv')

In [8]:
hotelinfo.head()

,hotel,totalrooms,isebookinghtl,hotelbelongto,supplierid,masterhotelid
0,8164,2,F,PKG,NaN,8164
1,19660,4,F,HTL,NaN,19660
2,21524,2,F,HTL,NaN,21524
3,21653,1,F,HTL,2654.0,994188
4,22486,2,F,HTL,NaN,22486


In [9]:
hotelinfo.rename(columns = {'totalrooms':'hotel_totalrooms'}, inplace=True)

In [10]:
data = pd.merge(data, hotelinfo[['hotel', 'hotel_totalrooms']], on = 'hotel', how = 'left')

In [11]:
del hotelinfo

In [12]:
data.head()

,arrival,city,commission,confirmdate,countryid,etd,hotel,hotelbelongto,hotelstar,isebookinghtl,...,eta_arrival_isholdroom_count,eta_arrival_masterbasicroomid_count,eta_arrival_masterhotelid_count,eta_arrival_supplierid_count,eta_arrival_isvendor_count,eta_arrival_hotelbelongto_count,eta_arrival_isebookinghtl_count,eta_arrival_supplierchannel_count,eta_arrival_count,hotel_totalrooms
0,2017-06-23,1777,57.60000,-1,111,2017-06-25,6459206,SHT,5,T,...,651836,82,588,26168,584516,527698,829851,577157,1069056,10.0
1,2017-05-05,622,7.98600,2017-05-02 21:25:45,4,2017-05-06,4515821,SHT,4,T,...,1082760,1,22,46917,873747,744915,1205202,770549,1669292,36.0
2,2017-07-01,366,89.71200,2017-06-20 15:03:53,30,2017-07-04,3403530,SHT,5,F,...,383423,20,147,5338,278917,328986,136322,365320,651342,119.0
3,2017-08-22,274,22.45260,2017-07-19 23:45:14,42,2017-08-24,4984070,HTL,5,T,...,417220,258,702,227340,584516,489356,829851,336875,1069056,166.0
4,2017-05-27,723,17.44395,-1,108,2017-05-28,3796626,SHT,5,T,...,1082760,181,205,591,795545,744915,1205202,770549,1669292,4.0


In [13]:
feature = 'hotel_totalrooms'
count_train = Counter(data[feature])
# count_test = Counter(data[data.noroom == -1][feature])
new_feature = feature + '_count'
data[new_feature] = data[feature].apply(lambda x: count_train[x])



In [14]:
del count_train
del new_feature

In [15]:
data.head()#总特征 368个

,arrival,city,commission,confirmdate,countryid,etd,hotel,hotelbelongto,hotelstar,isebookinghtl,...,eta_arrival_masterbasicroomid_count,eta_arrival_masterhotelid_count,eta_arrival_supplierid_count,eta_arrival_isvendor_count,eta_arrival_hotelbelongto_count,eta_arrival_isebookinghtl_count,eta_arrival_supplierchannel_count,eta_arrival_count,hotel_totalrooms,hotel_totalrooms_count
0,2017-06-23,1777,57.60000,-1,111,2017-06-25,6459206,SHT,5,T,...,82,588,26168,584516,527698,829851,577157,1069056,10.0,68982
1,2017-05-05,622,7.98600,2017-05-02 21:25:45,4,2017-05-06,4515821,SHT,4,T,...,1,22,46917,873747,744915,1205202,770549,1669292,36.0,38367
2,2017-07-01,366,89.71200,2017-06-20 15:03:53,30,2017-07-04,3403530,SHT,5,F,...,20,147,5338,278917,328986,136322,365320,651342,119.0,6191
3,2017-08-22,274,22.45260,2017-07-19 23:45:14,42,2017-08-24,4984070,HTL,5,T,...,258,702,227340,584516,489356,829851,336875,1069056,166.0,5692
4,2017-05-27,723,17.44395,-1,108,2017-05-28,3796626,SHT,5,T,...,181,205,591,795545,744915,1205202,770549,1669292,4.0,75412


In [16]:
list(data.columns)

['arrival',
 'city',
 'commission',
 'confirmdate',
 'countryid',
 'etd',
 'hotel',
 'hotelbelongto',
 'hotelstar',
 'isebookinghtl',
 'isholdroom',
 'isvendor',
 'masterbasicroomid',
 'masterhotelid',
 'noroom',
 'ordadvanceday',
 'orderdate',
 'orderid',
 'orderstatus',
 'ordroomnum',
 'price',
 'room',
 'supplierchannel',
 'supplierid',
 'zone',
 'time_arrival',
 'month_arrival',
 'week_arrival',
 'orderdate_hour',
 'city_count',
 'countryid_count',
 'hotel_count',
 'zone_count',
 'room_count',
 'isholdroom_count',
 'ordadvanceday_count',
 'arrival_count',
 'etd_count',
 'masterbasicroomid_count',
 'masterhotelid_count',
 'supplierid_count',
 'isvendor_count',
 'hotelbelongto_count',
 'isebookinghtl_count',
 'hotelstar_count',
 'supplierchannel_count',
 'time_arrival_count',
 'month_arrival_count',
 'week_arrival_count',
 'orderdate_hour_count',
 'eta_arrival',
 'cvr_ordadvanceday',
 'cvr_week_arrival',
 'arrival_day',
 'orderdate_day',
 'orderdate_mouth',
 'orderdate_week',
 'arriv

In [17]:
gc.collect()

420

In [37]:
len(data[(data.noroom == -1) & (data.room_count == 1)])

3349

In [75]:


one_hot_feature = [ # 'orderdate',
                   'arrival',
    'etd', # 1
    # row
                   'city',
    'countryid',
    'hotel',
    #'zone',
    'room',
    'isholdroom',
           'masterbasicroomid',
    'masterhotelid',
    'supplierid',
    'isvendor',
           'hotelbelongto',
    'isebookinghtl',
    'supplierchannel',
# arrival mouth week day
          'month_arrival', 
    #'roomstatus',
        #'week_arrival', 
    
          # 'arrival_day', 
    
           # orderdate mouth week day hour
        'orderdate_hour',
        'orderdate_day', # 1
    
     #  'orderdate_mouth',
     #   'orderdate_week',
    # 
    'week_arrival_room', # [1800]	valid_0's binary_logloss: 0.139355
 #'arrival_hotelstar',
 'arrival_city',
 'arrival_countryid',
 'arrival_hotel',
 'arrival_zone',
 'arrival_room',
 'arrival_isholdroom',
 'arrival_masterbasicroomid', 
 'arrival_masterhotelid', 
 'arrival_supplierid', 
 'arrival_isvendor', 
 'arrival_hotelbelongto', # 1
 'arrival_isebookinghtl',
 'arrival_supplierchannel', # 1
 'etd_hotelstar',
 'etd_city',
 'etd_countryid',
 'etd_hotel',
 'etd_zone',
 'etd_room',
 'etd_isholdroom',
 'etd_masterbasicroomid',
 'etd_masterhotelid',
 'etd_supplierid',
 'etd_isvendor',
 'etd_hotelbelongto',
 'etd_isebookinghtl',
 'etd_supplierchannel',
 'ordadvanceday_hotelstar',
 'ordadvanceday_city',
 'ordadvanceday_countryid',
 'ordadvanceday_hotel',
 'ordadvanceday_zone',
 'ordadvanceday_room',
 'ordadvanceday_isholdroom',
 'ordadvanceday_masterbasicroomid',
 'ordadvanceday_masterhotelid',
 'ordadvanceday_supplierid',
 'ordadvanceday_isvendor',
 'ordadvanceday_hotelbelongto',
 'ordadvanceday_isebookinghtl',
 'ordadvanceday_supplierchannel',
 'month_arrival_hotelstar',
 'month_arrival_city',
 'month_arrival_countryid',
 'month_arrival_hotel',
 'month_arrival_zone',
 'month_arrival_room',
 'month_arrival_isholdroom',
 'month_arrival_masterbasicroomid',
 'month_arrival_masterhotelid',
 'month_arrival_supplierid',
 'month_arrival_isvendor',
 'month_arrival_hotelbelongto',
 'month_arrival_isebookinghtl',
 'month_arrival_supplierchannel',
 'week_arrival_hotelstar',
 'week_arrival_city',
 'week_arrival_countryid',
 'week_arrival_hotel',
 'week_arrival_zone',
 
 'week_arrival_isholdroom',
 'week_arrival_masterbasicroomid',
 'week_arrival_masterhotelid',
 'week_arrival_supplierid',
 'week_arrival_isvendor',
 'week_arrival_hotelbelongto',
 'week_arrival_isebookinghtl',
 'week_arrival_supplierchannel',
 'arrival_day_hotelstar',
 'arrival_day_city',
 'arrival_day_countryid',
 'arrival_day_hotel',
 'arrival_day_zone',
 'arrival_day_room',
 'arrival_day_isholdroom',
 'arrival_day_masterbasicroomid',
 'arrival_day_masterhotelid',
 'arrival_day_supplierid',
 'arrival_day_isvendor',
 'arrival_day_hotelbelongto',
 'arrival_day_isebookinghtl',
 'arrival_day_supplierchannel',
 'orderdate_hour_hotelstar',
 'orderdate_hour_city',
 'orderdate_hour_countryid',
 'orderdate_hour_hotel',
 'orderdate_hour_zone',
 'orderdate_hour_room',
 'orderdate_hour_isholdroom',
 'orderdate_hour_masterbasicroomid',
 'orderdate_hour_masterhotelid',
 'orderdate_hour_supplierid',
 'orderdate_hour_isvendor',
 'orderdate_hour_hotelbelongto',
 'orderdate_hour_isebookinghtl',
 'orderdate_hour_supplierchannel',
 'orderdate_day_hotelstar',
 'orderdate_day_city',
 'orderdate_day_countryid',
 'orderdate_day_hotel',
 'orderdate_day_zone',
 'orderdate_day_room',
 'orderdate_day_isholdroom',
 'orderdate_day_masterbasicroomid',
 'orderdate_day_masterhotelid',
 'orderdate_day_supplierid',
 'orderdate_day_isvendor',
 'orderdate_day_hotelbelongto',
 'orderdate_day_isebookinghtl',
 'orderdate_day_supplierchannel',
 'orderdate_mouth_hotelstar',
 'orderdate_mouth_city',
 'orderdate_mouth_countryid',
 'orderdate_mouth_hotel',
 'orderdate_mouth_zone',
 'orderdate_mouth_room',
 'orderdate_mouth_isholdroom',
 'orderdate_mouth_masterbasicroomid',
 'orderdate_mouth_masterhotelid',
 'orderdate_mouth_supplierid',
 'orderdate_mouth_isvendor',
 'orderdate_mouth_hotelbelongto',
 'orderdate_mouth_isebookinghtl',
 'orderdate_mouth_supplierchannel',
 'orderdate_week_hotelstar',
 'orderdate_week_city',
 'orderdate_week_countryid',
 'orderdate_week_hotel',
 'orderdate_week_zone',
 'orderdate_week_room',
 'orderdate_week_isholdroom',
 'orderdate_week_masterbasicroomid',
 'orderdate_week_masterhotelid',
 'orderdate_week_supplierid',
 'orderdate_week_isvendor',
 'orderdate_week_hotelbelongto',
 'orderdate_week_isebookinghtl',
 'orderdate_week_supplierchannel',
 'eta_arrival_hotelstar',
 'eta_arrival_city',
 'eta_arrival_countryid',
 'eta_arrival_hotel',
 'eta_arrival_zone',
 'eta_arrival_room',
 'eta_arrival_isholdroom',
 'eta_arrival_masterbasicroomid',
 'eta_arrival_masterhotelid',
 'eta_arrival_supplierid',
 'eta_arrival_isvendor',
 'eta_arrival_hotelbelongto',
 'eta_arrival_isebookinghtl',
 'eta_arrival_supplierchannel',
]
    # 
cate = [
    # row
    'ordadvanceday',
    'hotelstar',
    # count
    'city_count', 
    'countryid_count', 
    'hotel_count', 
    'zone_count',
       'room_count', 
    'isholdroom_count', 
    'ordadvanceday_count',
       'arrival_count', 
    'etd_count', 
    'masterbasicroomid_count',
       'masterhotelid_count', 
    'supplierid_count', 
    'isvendor_count',
       'hotelbelongto_count',
    'isebookinghtl_count', 
    'hotelstar_count',
       'supplierchannel_count',  
        # 'time_arrival_count', 
        'month_arrival_count', 
       'week_arrival_count',  
    'orderdate_hour_count', 
         'time_arrival', 
    
        
    
    # 2000 0.139243
    
       # start - end
       'eta_arrival', # 0.139217 # 1
    
    #'hotel_totalrooms',
     #  'mroom_totalrooms',
    #'glon', 
    #'glat'
    'arrival_hotelstar_count',
 'arrival_city_count',
 'arrival_countryid_count',
 'arrival_hotel_count',
    'arrival_zone_count', # 1
 #'arrival_room_count', # 0
 'arrival_isholdroom_count',
 'arrival_masterbasicroomid_count',
 'arrival_masterhotelid_count',
 'arrival_supplierid_count',
 'arrival_isvendor_count',
 'arrival_hotelbelongto_count',
 'arrival_isebookinghtl_count',
 'arrival_supplierchannel_count',
 'etd_hotelstar_count',
 'etd_city_count',
 'etd_countryid_count',
 'etd_hotel_count',
 'etd_zone_count',
 'etd_room_count',
 'etd_isholdroom_count',
 'etd_masterbasicroomid_count',
 'etd_masterhotelid_count',
 'etd_supplierid_count',
 'etd_isvendor_count',
 'etd_hotelbelongto_count',
 'etd_isebookinghtl_count',
 'etd_supplierchannel_count',
 'ordadvanceday_hotelstar_count',
 'ordadvanceday_city_count',
 'ordadvanceday_countryid_count',
 'ordadvanceday_hotel_count',
 'ordadvanceday_zone_count',
 'ordadvanceday_room_count',
 'ordadvanceday_isholdroom_count',
 'ordadvanceday_masterbasicroomid_count',
 'ordadvanceday_masterhotelid_count',
 'ordadvanceday_supplierid_count',
 'ordadvanceday_isvendor_count',
 'ordadvanceday_hotelbelongto_count',
 'ordadvanceday_isebookinghtl_count',
 'ordadvanceday_supplierchannel_count',
 'month_arrival_hotelstar_count',
 'month_arrival_city_count',
 'month_arrival_countryid_count',
 'month_arrival_hotel_count',
 'month_arrival_zone_count',
 'month_arrival_room_count',
 'month_arrival_isholdroom_count',
 'month_arrival_masterbasicroomid_count',
 'month_arrival_masterhotelid_count',
 'month_arrival_supplierid_count',
 'month_arrival_isvendor_count',
 'month_arrival_hotelbelongto_count',
 'month_arrival_isebookinghtl_count',
 'month_arrival_supplierchannel_count',
 'week_arrival_hotelstar_count',
 'week_arrival_city_count',
 'week_arrival_countryid_count',
 'week_arrival_hotel_count',
 'week_arrival_zone_count',
 'week_arrival_room_count',
 'week_arrival_isholdroom_count',
 'week_arrival_masterbasicroomid_count',
 'week_arrival_masterhotelid_count',
 'week_arrival_supplierid_count',
 'week_arrival_isvendor_count',
 'week_arrival_hotelbelongto_count',
 'week_arrival_isebookinghtl_count',
 'week_arrival_supplierchannel_count',
 'arrival_day_hotelstar_count',
 'arrival_day_city_count',
 'arrival_day_countryid_count',
 'arrival_day_hotel_count',
 'arrival_day_zone_count',
 'arrival_day_room_count',
 'arrival_day_isholdroom_count',
 'arrival_day_masterbasicroomid_count',
 'arrival_day_masterhotelid_count',
 'arrival_day_supplierid_count',
 'arrival_day_isvendor_count',
 'arrival_day_hotelbelongto_count',
 'arrival_day_isebookinghtl_count',
 'arrival_day_supplierchannel_count',
 'orderdate_hour_hotelstar_count',
 'orderdate_hour_city_count',
 'orderdate_hour_countryid_count',
 'orderdate_hour_hotel_count',
 'orderdate_hour_zone_count',
 'orderdate_hour_room_count',
 'orderdate_hour_isholdroom_count',
 'orderdate_hour_masterbasicroomid_count',
 'orderdate_hour_masterhotelid_count',
 'orderdate_hour_supplierid_count',
 'orderdate_hour_isvendor_count',
 'orderdate_hour_hotelbelongto_count',
 'orderdate_hour_isebookinghtl_count',
 'orderdate_hour_supplierchannel_count',
 'orderdate_day_hotelstar_count',
 'orderdate_day_city_count',
 'orderdate_day_countryid_count',
 'orderdate_day_hotel_count',
 'orderdate_day_zone_count',
 'orderdate_day_room_count',
 'orderdate_day_isholdroom_count',
 'orderdate_day_masterbasicroomid_count',
 'orderdate_day_masterhotelid_count',
 'orderdate_day_supplierid_count',
 'orderdate_day_isvendor_count',
 'orderdate_day_hotelbelongto_count',
 'orderdate_day_isebookinghtl_count',
 'orderdate_day_supplierchannel_count',
 'orderdate_mouth_hotelstar_count',
 'orderdate_mouth_city_count',
 'orderdate_mouth_countryid_count',
 'orderdate_mouth_hotel_count',
 'orderdate_mouth_zone_count',
 'orderdate_mouth_room_count',
 'orderdate_mouth_isholdroom_count',
 'orderdate_mouth_masterbasicroomid_count',
 'orderdate_mouth_masterhotelid_count',
 'orderdate_mouth_supplierid_count',
 'orderdate_mouth_isvendor_count',
 'orderdate_mouth_hotelbelongto_count',
 'orderdate_mouth_isebookinghtl_count',
 'orderdate_mouth_supplierchannel_count',
 'orderdate_week_hotelstar_count',
 'orderdate_week_city_count',
 'orderdate_week_countryid_count',
 'orderdate_week_hotel_count',
 'orderdate_week_zone_count',
 'orderdate_week_room_count',
 'orderdate_week_isholdroom_count',
 'orderdate_week_masterbasicroomid_count',
 'orderdate_week_masterhotelid_count',
 'orderdate_week_supplierid_count',
 'orderdate_week_isvendor_count',
 'orderdate_week_hotelbelongto_count',
 'orderdate_week_isebookinghtl_count',
 'orderdate_week_supplierchannel_count',
    
 'eta_arrival_hotelstar_count',
 'eta_arrival_city_count',
 'eta_arrival_countryid_count',
 'eta_arrival_hotel_count',
 'eta_arrival_zone_count',
 'eta_arrival_room_count',
 'eta_arrival_isholdroom_count',
 'eta_arrival_masterbasicroomid_count',
 'eta_arrival_masterhotelid_count',
 'eta_arrival_supplierid_count',
 'eta_arrival_isvendor_count',
 'eta_arrival_hotelbelongto_count',
 'eta_arrival_isebookinghtl_count',
 'eta_arrival_supplierchannel_count',
    
    'eta_arrival_count', # 1
    'hotel_totalrooms', # 1
 'hotel_totalrooms_count', # 1

    
       ]

cvr = [
    


    
    
]

In [17]:
for feature in [
    
    'arrival',
    'etd',
    #'roomstatus',
    # row
    'city','countryid','hotel',
    # 'zone',
    'room',
    'isholdroom',
        'masterbasicroomid',
    'masterhotelid',
    'supplierid',
    'isvendor',
           'hotelbelongto','isebookinghtl','supplierchannel',
# arrival mouth week day
          'month_arrival', 
        'week_arrival', 
           'arrival_day',
           # orderdate mouth week day hour
        'orderdate_hour',
        'orderdate_day',
       'orderdate_mouth',
        'orderdate_week']:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [18]:
gc.collect()

431

In [76]:
train=data[data.noroom !=-1]
train_y=train.pop('noroom')
test=data[data.noroom == -1]

test=test.drop('noroom',axis=1)

print(' success!')

'''
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a = enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a))
        
    del train_a
    del test_a
    gc.collect()
        
    print(feature+' finish')
print('one-hot prepared !')
'''

clf = lgb.LGBMClassifier(
        boosting_type='gbdt', subsample=0.70, colsample_bytree=0.70, 
        max_depth=-1, n_estimators=1800, objective='binary',min_child_weight = 30, 
        subsample_freq=1, num_leaves=31, reg_alpha=0,reg_lambda = 1,
         random_state=2018, n_jobs=-1, learning_rate=0.05
    )

from sklearn.metrics import confusion_matrix

def pr(preds, label):
    # print(preds)
    # print(label)
    dtrain = [int(i>=0.5) for i in label]
    confusion_matrixs = confusion_matrix(dtrain, preds)
    recall = float(confusion_matrixs[0][0] ) / float(confusion_matrixs[0][1] + confusion_matrixs[0][0])
    precision = float(confusion_matrixs[0][0]) / float(confusion_matrixs[1][0] + confusion_matrixs[0][0])
    return 'xiecheng', float(2*recall*precision)/(recall+precision), True


#clf.fit(train_x, train_y, eval_set=[(train_x, train_y)], eval_metric='logloss',early_stopping_rounds=100)
# [2000]	valid_0's binary_logloss: 0.13928
# [1800]	valid_0's binary_logloss: 0.137545 2503
# [1800]	valid_0's binary_logloss: 0.138594 2506
# [1800]	valid_0's binary_logloss: 0.133141 254

 success!


In [77]:
train_x = train[cate+one_hot_feature+cvr]
test_x = test[cate+one_hot_feature+cvr]

del train
del test
gc.collect()

X_train, X_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.3, random_state=2018)
clf.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='logloss', early_stopping_rounds=100)

[1]	valid_0's binary_logloss: 0.652573
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's binary_logloss: 0.61594
[3]	valid_0's binary_logloss: 0.582544
[4]	valid_0's binary_logloss: 0.552114
[5]	valid_0's binary_logloss: 0.524292
[6]	valid_0's binary_logloss: 0.498734
[7]	valid_0's binary_logloss: 0.475253
[8]	valid_0's binary_logloss: 0.453595
[9]	valid_0's binary_logloss: 0.433583
[10]	valid_0's binary_logloss: 0.41509
[11]	valid_0's binary_logloss: 0.397984
[12]	valid_0's binary_logloss: 0.382089
[13]	valid_0's binary_logloss: 0.367355
[14]	valid_0's binary_logloss: 0.353621
[15]	valid_0's binary_logloss: 0.340822
[16]	valid_0's binary_logloss: 0.328892
[17]	valid_0's binary_logloss: 0.317772
[18]	valid_0's binary_logloss: 0.307412
[19]	valid_0's binary_logloss: 0.297724
[20]	valid_0's binary_logloss: 0.288679
[21]	valid_0's binary_logloss: 0.280217
[22]	valid_0's binary_logloss: 0.272303
[23]	valid_0's binary_logloss: 0.264868
[24]	valid_0's binary_loglos

[203]	valid_0's binary_logloss: 0.14267
[204]	valid_0's binary_logloss: 0.142643
[205]	valid_0's binary_logloss: 0.142626
[206]	valid_0's binary_logloss: 0.142594
[207]	valid_0's binary_logloss: 0.142566
[208]	valid_0's binary_logloss: 0.142532
[209]	valid_0's binary_logloss: 0.142503
[210]	valid_0's binary_logloss: 0.142478
[211]	valid_0's binary_logloss: 0.142457
[212]	valid_0's binary_logloss: 0.142431
[213]	valid_0's binary_logloss: 0.142399
[214]	valid_0's binary_logloss: 0.142357
[215]	valid_0's binary_logloss: 0.142325
[216]	valid_0's binary_logloss: 0.142298
[217]	valid_0's binary_logloss: 0.142268
[218]	valid_0's binary_logloss: 0.142244
[219]	valid_0's binary_logloss: 0.142218
[220]	valid_0's binary_logloss: 0.142191
[221]	valid_0's binary_logloss: 0.142157
[222]	valid_0's binary_logloss: 0.142138
[223]	valid_0's binary_logloss: 0.142093
[224]	valid_0's binary_logloss: 0.142064
[225]	valid_0's binary_logloss: 0.142048
[226]	valid_0's binary_logloss: 0.142022
[227]	valid_0's b

[404]	valid_0's binary_logloss: 0.138891
[405]	valid_0's binary_logloss: 0.138886
[406]	valid_0's binary_logloss: 0.13887
[407]	valid_0's binary_logloss: 0.138854
[408]	valid_0's binary_logloss: 0.138845
[409]	valid_0's binary_logloss: 0.138835
[410]	valid_0's binary_logloss: 0.138815
[411]	valid_0's binary_logloss: 0.13881
[412]	valid_0's binary_logloss: 0.138799
[413]	valid_0's binary_logloss: 0.138784
[414]	valid_0's binary_logloss: 0.138771
[415]	valid_0's binary_logloss: 0.138749
[416]	valid_0's binary_logloss: 0.138737
[417]	valid_0's binary_logloss: 0.138721
[418]	valid_0's binary_logloss: 0.138711
[419]	valid_0's binary_logloss: 0.138703
[420]	valid_0's binary_logloss: 0.138697
[421]	valid_0's binary_logloss: 0.138683
[422]	valid_0's binary_logloss: 0.138677
[423]	valid_0's binary_logloss: 0.138664
[424]	valid_0's binary_logloss: 0.138657
[425]	valid_0's binary_logloss: 0.138651
[426]	valid_0's binary_logloss: 0.13864
[427]	valid_0's binary_logloss: 0.138635
[428]	valid_0's bin

[605]	valid_0's binary_logloss: 0.137103
[606]	valid_0's binary_logloss: 0.137098
[607]	valid_0's binary_logloss: 0.137074
[608]	valid_0's binary_logloss: 0.137067
[609]	valid_0's binary_logloss: 0.137062
[610]	valid_0's binary_logloss: 0.137051
[611]	valid_0's binary_logloss: 0.137046
[612]	valid_0's binary_logloss: 0.137035
[613]	valid_0's binary_logloss: 0.137034
[614]	valid_0's binary_logloss: 0.137019
[615]	valid_0's binary_logloss: 0.137011
[616]	valid_0's binary_logloss: 0.137004
[617]	valid_0's binary_logloss: 0.137004
[618]	valid_0's binary_logloss: 0.136999
[619]	valid_0's binary_logloss: 0.136997
[620]	valid_0's binary_logloss: 0.136993
[621]	valid_0's binary_logloss: 0.136975
[622]	valid_0's binary_logloss: 0.136969
[623]	valid_0's binary_logloss: 0.136964
[624]	valid_0's binary_logloss: 0.13696
[625]	valid_0's binary_logloss: 0.136956
[626]	valid_0's binary_logloss: 0.136955
[627]	valid_0's binary_logloss: 0.136952
[628]	valid_0's binary_logloss: 0.13695
[629]	valid_0's bi

[806]	valid_0's binary_logloss: 0.13596
[807]	valid_0's binary_logloss: 0.13595
[808]	valid_0's binary_logloss: 0.135947
[809]	valid_0's binary_logloss: 0.135942
[810]	valid_0's binary_logloss: 0.135938
[811]	valid_0's binary_logloss: 0.135934
[812]	valid_0's binary_logloss: 0.135932
[813]	valid_0's binary_logloss: 0.135928
[814]	valid_0's binary_logloss: 0.135923
[815]	valid_0's binary_logloss: 0.135919
[816]	valid_0's binary_logloss: 0.135916
[817]	valid_0's binary_logloss: 0.135914
[818]	valid_0's binary_logloss: 0.135904
[819]	valid_0's binary_logloss: 0.135903
[820]	valid_0's binary_logloss: 0.1359
[821]	valid_0's binary_logloss: 0.135896
[822]	valid_0's binary_logloss: 0.135893
[823]	valid_0's binary_logloss: 0.135889
[824]	valid_0's binary_logloss: 0.135886
[825]	valid_0's binary_logloss: 0.135883
[826]	valid_0's binary_logloss: 0.13588
[827]	valid_0's binary_logloss: 0.135878
[828]	valid_0's binary_logloss: 0.135874
[829]	valid_0's binary_logloss: 0.13587
[830]	valid_0's binary

...

[1203]	valid_0's binary_logloss: 0.134466
[1204]	valid_0's binary_logloss: 0.134462
[1205]	valid_0's binary_logloss: 0.134459
[1206]	valid_0's binary_logloss: 0.134455
[1207]	valid_0's binary_logloss: 0.134449
[1208]	valid_0's binary_logloss: 0.134448
[1209]	valid_0's binary_logloss: 0.134446
[1210]	valid_0's binary_logloss: 0.134445
[1211]	valid_0's binary_logloss: 0.134441
[1212]	valid_0's binary_logloss: 0.134438
[1213]	valid_0's binary_logloss: 0.134434
[1214]	valid_0's binary_logloss: 0.134426
[1215]	valid_0's binary_logloss: 0.134424
[1216]	valid_0's binary_logloss: 0.134422
[1217]	valid_0's binary_logloss: 0.134422
[1218]	valid_0's binary_logloss: 0.134418
[1219]	valid_0's binary_logloss: 0.134414
[1220]	valid_0's binary_logloss: 0.134411
[1221]	valid_0's binary_logloss: 0.134408
[1222]	valid_0's binary_logloss: 0.134401
[1223]	valid_0's binary_logloss: 0.134398
[1224]	valid_0's binary_logloss: 0.134398
[1225]	valid_0's binary_logloss: 0.134393
[1226]	valid_0's binary_logloss: 0

[1399]	valid_0's binary_logloss: 0.1339
[1400]	valid_0's binary_logloss: 0.133899
[1401]	valid_0's binary_logloss: 0.133898
[1402]	valid_0's binary_logloss: 0.133897
[1403]	valid_0's binary_logloss: 0.133894
[1404]	valid_0's binary_logloss: 0.13389
[1405]	valid_0's binary_logloss: 0.133885
[1406]	valid_0's binary_logloss: 0.133874
[1407]	valid_0's binary_logloss: 0.133871
[1408]	valid_0's binary_logloss: 0.13387
[1409]	valid_0's binary_logloss: 0.133868
[1410]	valid_0's binary_logloss: 0.133865
[1411]	valid_0's binary_logloss: 0.133861
[1412]	valid_0's binary_logloss: 0.133858
[1413]	valid_0's binary_logloss: 0.133855
[1414]	valid_0's binary_logloss: 0.133854
[1415]	valid_0's binary_logloss: 0.13385
[1416]	valid_0's binary_logloss: 0.133849
[1417]	valid_0's binary_logloss: 0.133849
[1418]	valid_0's binary_logloss: 0.133846
[1419]	valid_0's binary_logloss: 0.133843
[1420]	valid_0's binary_logloss: 0.133843
[1421]	valid_0's binary_logloss: 0.133838
[1422]	valid_0's binary_logloss: 0.1338

[1595]	valid_0's binary_logloss: 0.133415
[1596]	valid_0's binary_logloss: 0.13341
[1597]	valid_0's binary_logloss: 0.133408
[1598]	valid_0's binary_logloss: 0.133405
[1599]	valid_0's binary_logloss: 0.133405
[1600]	valid_0's binary_logloss: 0.133404
[1601]	valid_0's binary_logloss: 0.1334
[1602]	valid_0's binary_logloss: 0.133399
[1603]	valid_0's binary_logloss: 0.133397
[1604]	valid_0's binary_logloss: 0.133396
[1605]	valid_0's binary_logloss: 0.133394
[1606]	valid_0's binary_logloss: 0.133393
[1607]	valid_0's binary_logloss: 0.133392
[1608]	valid_0's binary_logloss: 0.133385
[1609]	valid_0's binary_logloss: 0.133383
[1610]	valid_0's binary_logloss: 0.133381
[1611]	valid_0's binary_logloss: 0.13338
[1612]	valid_0's binary_logloss: 0.133377
[1613]	valid_0's binary_logloss: 0.133374
[1614]	valid_0's binary_logloss: 0.133371
[1615]	valid_0's binary_logloss: 0.133362
[1616]	valid_0's binary_logloss: 0.133361
[1617]	valid_0's binary_logloss: 0.133361
[1618]	valid_0's binary_logloss: 0.133

[1791]	valid_0's binary_logloss: 0.133053
[1792]	valid_0's binary_logloss: 0.133051
[1793]	valid_0's binary_logloss: 0.13305
[1794]	valid_0's binary_logloss: 0.133048
[1795]	valid_0's binary_logloss: 0.133047
[1796]	valid_0's binary_logloss: 0.133044
[1797]	valid_0's binary_logloss: 0.133043
[1798]	valid_0's binary_logloss: 0.133042
[1799]	valid_0's binary_logloss: 0.133041
[1800]	valid_0's binary_logloss: 0.13304
Did not meet early stopping. Best iteration is:
[1800]	valid_0's binary_logloss: 0.13304


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=30, min_split_gain=0.0, n_estimators=1800,
        n_jobs=-1, num_leaves=31, objective='binary', random_state=2018,
        reg_alpha=0, reg_lambda=1, silent=True, subsample=0.7,
        subsample_for_bin=200000, subsample_freq=1)

In [78]:
gc.collect()

82

In [79]:
clf.feature_importances_

array([ 153,  152,  296,  245,  947,  182,  573,  156,   29,   91,   86,
        430,  483,  705,  158,    8,   26,   72,   68,    3,   25,   49,
         16,    5,  200,  355,  265,  240,  205,  145,  219,  344,  207,
        131,  150,  190,  159,  207,  333,  287,  212,  198,  349,  151,
        164,  363,  240,   85,  156,  188,  170,  250,  288,  402,  231,
        308,  215,  151,  206,  365,  417,  185,  161,  282,  226,  139,
        275,  265,  384,  176,  283,   52,  277,  322,  313,   47,   82,
         60,   86,  108,  242,  243,  203,  122,  115,  119,  148,  227,
        243,  107,   71,   54,   93,  219,  168,  304,   93,  162,  131,
        183,  121,  160,  216,  163,  144,  162,  173,  156,  190,  198,
        143,  151,  100,   92,  136,  133,  200,  112,  103,  104,  123,
        252,  176,  270,  258,  167,  170,  190,  165,  169,  296,  184,
        174,  230,  243,  195,  315,  272,  485,  191,  260,  133,  246,
        362,  573,  127,   80,  176,  222,  133,  1

In [80]:
testa = pd.read_csv('./data/test/ord_testA.csv', encoding='gb2312')
testb = pd.read_csv('./data/test/ord_testB.csv', encoding='gb2312')

pre = pd.concat([testa, testb])
pre = pre[['orderid','room','arrival']]

pre['noroom'] = clf.predict_proba(test_x)[:,1]
pre.to_csv('xj6.csv', index = None)

testa = testa[['orderid','room','arrival']]

pre_a = pd.merge(testa, pre, on = ['orderid','room','arrival'], how = 'left')

pre_a.to_csv('xj6.csv', index = None)

In [1]:
3000-4000//5000    7000-7500

SyntaxError: invalid syntax (<ipython-input-1-7138138136d8>, line 1)